## R code testing prototypes


filtered counts: gs://asap-raw-data-team-lee/cellranger_counts/SN/SN_0009_PD_count_filtered_feature_bc_matrix.h5


raw counts: gs://asap-raw-data-team-lee/cellranger_counts/SN/SN_0009_PD_count_raw_feature_bc_matrix.h5




In [2]:

install.packages("Seurat")



also installing the dependencies ‘rmarkdown’, ‘htmlwidgets’, ‘tidyr’, ‘gplots’, ‘RcppArmadillo’, ‘httpuv’, ‘bslib’, ‘spatstat.data’, ‘spatstat.random’, ‘spatstat.sparse’, ‘dqrng’, ‘cowplot’, ‘future’, ‘future.apply’, ‘ggplot2’, ‘ggrepel’, ‘ggridges’, ‘leiden’, ‘lmtest’, ‘miniUI’, ‘patchwork’, ‘plotly’, ‘reticulate’, ‘ROCR’, ‘scales’, ‘scattermore’, ‘sctransform’, ‘SeuratObject’, ‘shiny’, ‘spatstat.explore’, ‘spatstat.geom’, ‘uwot’, ‘RcppEigen’


Warning message in install.packages("Seurat"):
“installation of package ‘RcppArmadillo’ had non-zero exit status”
Warning message in install.packages("Seurat"):
“installation of package ‘httpuv’ had non-zero exit status”
Warning message in install.packages("Seurat"):
“installation of package ‘RcppEigen’ had non-zero exit status”
Warning message in install.packages("Seurat"):
“installation of package ‘shiny’ had non-zero exit status”


In [ ]:

install.packages("reticulate")
install.packages("anndata")
install.packages("devtools")
devtools::install_github("satijalab/seurat-data")

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install(c("LoomExperiment", "SingleCellExperiment"))
devtools::install_github("cellgeni/sceasy")



In [1]:

library(Seurat)
library(SeuratData)
library(ggplot2)



ERROR: Error in library(Seurat): there is no package called ‘Seurat’


In [ ]:

input_table = read.csv('team-Lee-samples.csv')

datasets = pandas.read_csv(input_table, header=None).loc[:, 0].tolist()[1:]


In [ ]:

dataset <- 'team-Lee-samples.csv'


batch <- fread(snakemake@input[['datasets']])[sample %chin% dataset,]$batch

raw.counts <- Read10X_h5(paste0(data_path, batch, '/Multiome/', dataset, '/outs/raw_feature_bc_matrix.h5'))
filtered.counts <- Read10X_h5(paste0(data_path, batch, '/Multiome/', dataset, '/outs/filtered_feature_bc_matrix.h5'))

adj.matrix <- suppressWarnings(SoupCorrect(raw.counts, filtered.counts, contamination_rate=snakemake@params[['soup_rate']]))
object <- CreateSeuratObject(adj.matrix, min.cells=0, min.features=0, project=dataset)


object[['percent.mt']] <- PercentageFeatureSet(object, pattern='^MT-')
object[['percent.rb']] <- PercentageFeatureSet(object, pattern='^RP[SL]')

m <- copy(object@meta.data)
setDT(m, keep.rownames='cells')

m[,
    `:=` (
        sample=dataset,
        batch=batch
        )
]

batch <- m[, batch]
sample <- m[, sample]

names(sample) <- names(batch) <- m[, cells]


doublet_rate <- (ncol(object) / 1000) * 0.008

object <- object %>% 
    
    AddMetaData(metadata=factor(batch), col.name='batch') %>% 
    AddMetaData(metadata=factor(sample), col.name='sample') %>% 
    
    scrublet(n_prin_comps=30, expected_doublet_rate=doublet_rate) 


saveRDS(object, snakemake@output[['seurat_object']])